In [1]:
from collections import defaultdict
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset

np.random.seed(234)
torch.manual_seed(234)

/Users/mohit/opt/anaconda3/envs/exp_pytorch_only/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test = np.array([0, 1, 2, 5, 0])
states = [0, 2]
mask = np.in1d(test, states)
print(mask)
mask1 = np.in1d(test, states,invert=True)
print(mask1)
mask2= np.in1d(test, states,assume_unique=True)
print(mask2)
mask3 = np.in1d(test, states,assume_unique=False)
print(mask3)

[ True False  True False  True]
[False  True False  True False]
[ True False  True False  True]
[ True False  True False  True]


In [3]:
p=[(0,11),(1,12),(0,13),(0,14)]
p=torch.LongTensor(p)
print(p)

q=p[:,0]
print(q)

r=p[:,1]
print(r)

d=[(0,21),(1,22),(0,23),(0,24)]
d=torch.LongTensor(d)
print(d)

e=(p[:,0],d[:,0])
print(e)
print(type(e))

tensor([[ 0, 11],
        [ 1, 12],
        [ 0, 13],
        [ 0, 14]])
tensor([0, 1, 0, 0])
tensor([11, 12, 13, 14])
tensor([[ 0, 21],
        [ 1, 22],
        [ 0, 23],
        [ 0, 24]])
(tensor([0, 1, 0, 0]), tensor([0, 1, 0, 0]))
<class 'tuple'>


In [4]:
a=defaultdict(set)
b=defaultdict(set)
print(a)
a[(1,2)].add(3)
print(a)
print(a['a'])

defaultdict(<class 'set'>, {})
defaultdict(<class 'set'>, {(1, 2): {3}})
set()


In [5]:
r=torch.randint(100,size=(3,))
print(r)

tensor([20, 91, 88])


In [6]:
class DataGenerator(Dataset):
    def __init__(self, triples, num_entities, num_negative_samples, all_triples=None, data_type="train"):
        super(Dataset, self).__init__()
        self.triples = triples
        self.num_entities = num_entities
        self.num_negative_samples = num_negative_samples
        self.all_triples = all_triples
        self.data_type = data_type
        self.len = len(triples)
        self.true_head_relation, self.true_relation_tail = self._get_true_head_tail_lists()

     
    def __len__(self):
        return self.len
    
    

    
    def __getitem__(self, idx):
        positive_sample=self.triples[idx]
        head, relation, tail = positive_sample
        positive_sample=torch.LongTensor(positive_sample)
        
        if self.data_type == 'train':
            true_heads=self.true_relation_tail[(relation, tail)]
            true_tails=self.true_head_relation[(head, relation)]
            
            corrupted_heads=[]
            corrupted=np.random.randint(self.num_entities, size=self.num_negative_samples*2)
            while len(corrupted_heads) < self.num_negative_samples:
                mask=np.in1d(corrupted, true_heads, assume_unique=True, invert=True)
                corrupted=corrupted[mask]
                corrupted_heads.extend(corrupted)
            corrupted_heads=corrupted_heads[:self.num_negative_samples]
            corrupted_heads=torch.LongTensor(corrupted_heads)
            
            corrupted_tails=[]
            corrupted=np.random.randint(self.num_entities, size=self.num_negative_samples*2)
            while len(corrupted_tails) < self.num_negative_samples:
                mask=np.in1d(corrupted, true_tails, assume_unique=True, invert=True)
                corrupted=corrupted[mask]
                corrupted_tails.extend(corrupted)
            corrupted_tails=corrupted_tails[:self.num_negative_samples]
            corrupted_tails=torch.LongTensor(corrupted_tails)
            
            filter_bias=torch.LongTensor([0]*len(positive_sample))
        
        else:
            corrupted_heads=[(0, test_head) if (test_head, relation, tail) not in self.all_triples
                            else (-1, head) for test_head in range(self.num_entities)]
            corrupted_heads[head]=(0,head)
            corrupted_heads=torch.LongTensor(corrupted_heads)
            
            corrupted_tails=[(0, test_tail) if (head, relation, test_tail) not in self.all_triples
                            else (-1, tail) for test_tail in range(self.num_entities)]
            corrupted_tails[tail]=(0,tail)
            corrupted_tails=torch.LongTensor(corrupted_tails)
            
            filter_bias=(corrupted_heads[:,0], corrupted_tails[:,0])
            corrupted_heads=corrupted_heads[:,1]
            corrupted_tails=corrupted_tails[:,1]
        
        return positive_sample, corrupted_heads, corrupted_tails, filter_bias    
    
    def _get_true_head_tail_lists(self):
        true_head_relation=defaultdict(set)
        true_relation_tail=defaultdict(set)
        for triplet in self.triples:
            head, relation, tail = triplet
            true_head_relation[(head, relation)].add(tail)
            true_relation_tail[(relation, tail)].add(head)
        return true_head_relation, true_relation_tail    

In [7]:
def get_data_loader(triples, num_entities, num_negative_samples, batch_size,
                    all_triples=None, data_type='train', num_workers=2):
    
    return DataLoader(
        DataGenerator(triples, num_entities, num_negative_samples, all_triples=all_triples, data_type=data_type),
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    )

In [8]:
from torch import linalg as LA 
    
c = torch.tensor([[1., 2., 3.],
                 [-1, 1, 4]])
print(c)
print(LA.norm(c, ord=1, dim=0))
print(LA.norm(c, ord=1, dim=1))
# print(LA.norm(c, ord=1, dim=2))
print(LA.norm(c, ord=2, dim=0))
print(LA.norm(c, ord=2, dim=1))

tensor([[ 1.,  2.,  3.],
        [-1.,  1.,  4.]])
tensor([2., 3., 7.])
tensor([6., 6.])
tensor([1.4142, 2.2361, 5.0000])
tensor([3.7417, 4.2426])


In [9]:
e1 = torch.randint(3,5,(4, 4))
print(e1)
print('e1 size ',e1.size())
z= torch.tensor([1, 0])

print(z)
h1 = torch.index_select(
          e1,
          dim=0,
          index=z).unsqueeze(1)


print(h1.size())
print(h1)

h2 = torch.index_select(
          e1,
          dim=0,
          index=z)

print(h2.size())
print(h2)

h3 = torch.index_select(
          e1,
          dim=0,
          index=z).unsqueeze(0)


print(h3.size())
print(h3)

h4 = torch.index_select(
          e1,
          dim=0,
          index=z).unsqueeze(2)


print(h4.size())
print(h4)

h5 = torch.index_select(
          e1,
          dim=0,
          index=z).unsqueeze(-3)


print(h5.size())
print(h5)

z1=torch.tensor([[1, 0],[2,3]])
print(z1.size())
print(z1.view(-1).size())

tensor([[4, 4, 4, 4],
        [4, 3, 4, 3],
        [4, 4, 4, 3],
        [3, 3, 3, 3]])
e1 size  torch.Size([4, 4])
tensor([1, 0])
torch.Size([2, 1, 4])
tensor([[[4, 3, 4, 3]],

        [[4, 4, 4, 4]]])
torch.Size([2, 4])
tensor([[4, 3, 4, 3],
        [4, 4, 4, 4]])
torch.Size([1, 2, 4])
tensor([[[4, 3, 4, 3],
         [4, 4, 4, 4]]])
torch.Size([2, 4, 1])
tensor([[[4],
         [3],
         [4],
         [3]],

        [[4],
         [4],
         [4],
         [4]]])
torch.Size([1, 2, 4])
tensor([[[4, 3, 4, 3],
         [4, 4, 4, 4]]])
torch.Size([2, 2])
torch.Size([4])


In [10]:
def transE(head, relation, tail, sample_type, margin):
    dist=head + relation - tail
    score=margin- torch.linalg.norm(dist, ord=1, dim=2)
    return score
    

In [11]:
import math
import torch.nn as nn
import torch.nn.functional as F

In [12]:
class KGEmbedding(nn.Module):
    def __init__(self, num_entities, num_relations, hidden_dim, margin, model):
        super(KGEmbedding, self).__init__()
        self.num_entities=num_entities
        self.num_relations=num_relations
    
        self.entity_dim=2*hidden_dim if model == 'rotatE' else hidden_dim 
        self.relation_dim=hidden_dim
        self.margin=margin
        self.model=model
        self.epsilon=2.0
        
        self.embedding_range= (self.margin + self.epsilon)/hidden_dim
        
        self.entity_embed = nn.Parameter(torch.zeros(self.num_entities, self.entity_dim,
                                                    requires_grad=True))
        
        nn.init.uniform_(
            self.entity_embed,
            a=-self.embedding_range,
            b=self.embedding_range
        )
        
        self.relation_embed=nn.Parameter(torch.zeros(self.num_relations, self.relation_dim,
                                                    requires_grad=True))
        
        nn.init.uniform_(
            self.relation_embed,
            a=-self.embedding_range,
            b=self.embedding_range
        )
    
    def forward(self, sample, sample_type):
        
        if sample_type=='positive':
            head=torch.index_select(
                self.entity_embed,
                dim=0,
                index=sample[:,0]
            ).unsqueeze(1)
            
            relation=torch.index_select(
                self.relation_embed,
                dim=0,
                index=sample[:,1]
            ).unsqueeze(1)
            
            tail=torch.index_select(
                self.entity_embed,
                dim=0,
                index=sample[:,2]
            ).unsqueeze(1)
            
        elif sample_type=='negative-head':
            
            positive_tuple, negative_head_entities = sample
            batch_size, num_neg_samples = negative_head_entities.shape
            
            head=torch.index_select(
                self.entity_embed,
                dim=0,
                index=negative_head_entities.view(-1)
            ).reshape(batch_size, num_neg_samples, self.entity_dim)
            
            relation = torch.index_select(
                    self.relation_embed,
                    dim=0,
                    index=positive_tuple[:,1]
            ).unsqueeze(1)
            
            tail=torch.index_select(
                self.entity_embed,
                dim=0,
                index=positive_tuple[:,2]
            ).unsqueeze(1)
            
        elif sample_type == 'negative-tail':
            positive_tuple, negative_tail_entities = sample
            batch_size, num_neg_samples = negative_tail_entities.shape
            
            head=torch.index_select(
                self.entity_embed,
                dim=0,
                index=positive_tuple[:,0]
            ).unsqueeze(1)
            
            relation=torch.index_select(
                self.relation_embed,
                dim=0,
                index=positive_tuple[:,1]
            ).unsqueeze(1)
            
            tail=torch.index_select(
                self.entity_embed,
                dim=0,
                index=negative_tail_entities.view(-1)
            ).reshape(batch_size, num_neg_samples, self.entity_dim)
            
         
        if self.model == 'rotatE':
            return rotatE(head, relation, tail, self.embedding_range, sample_type, self.margin)
        elif self.model == 'transE':
            return transE(head, relation, tail, sample_type, self.margin)
            
            
        
            

In [13]:
import os
from tqdm import trange

DATA_DIR = './FB15k-237/'
MODEL_DIR='models/'

In [14]:
s='  hello  '
print(s.strip()+'abc')

w='  abc\tdef pq  '
print(w)
w1=w.strip()
print(w1)
w2=w1.split('\t')
print(w2)

print('checkpoint_{3}')

helloabc
  abc	def pq  
abc	def pq
['abc', 'def pq']
checkpoint_{3}


In [15]:
def load_dict(file_path):
    loaded_dict=dict()
    with open(file_path, 'r') as f:
        for line in f:
            uid, val = line.strip().split('\t')
            loaded_dict[val]=int(uid)
    
    return loaded_dict         
            

In [16]:
def load_triples(file_path, entity2id, relation2id):
    triples=list()
    with open(file_path, 'r') as f:
        for line in f:
            head, relation, tail = line.strip().split('\t')
            triples.append((entity2id[head], relation2id[relation], entity2id[tail]))
    return triples        

In [17]:
def save_model(model, optimizer, scheduler, epoch):
    if not os.path.exists(MODEL_DIR):
        os.makedirs(MODEL_DIR)
        
    torch.save({
        'model_state_dict':model.state_dict(),
        'optimizer_state_dict':optimizer.state_dict(),
        'scheduler_state_dict':scheduler.state_dict()
    },
        os.path.join(MODEL_DIR,f'checkpoint_{epoch}')
    )
    
    entity_embedding=model.entity_embed.detach().cpu().numpy()
    np.save(os.path.join(MODEL_DIR, f'entity_embedding_{epoch}'),entity_embedding)
    
    relation_embedding=model.relation_embed.detach().cpu().numpy()
    np.save(os.path.join(MODEL_DIR, f'relation_embedding_{epoch}'), relation_embedding)

In [43]:
a = torch.arange(16).view(-1,4)
print(a)

z= torch.argsort(a,dim=0,descending=True)
print(z)

q2=5
print('test_{q2}')
print(f'test_{q2}')

for i in trange(z.shape[0]):
    print(i)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])
tensor([[3, 3, 3, 3],
        [2, 2, 2, 2],
        [1, 1, 1, 1],
        [0, 0, 0, 0]])
test_{q2}
test_5


100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 16794.01it/s]

0
1
2
3


In [44]:
print(torch.cuda.is_available())

False


In [45]:
def eval_model(model, data, num_entities, _num_negative_samples, _batch_size, all_data, data_type):
    model.eval()
    
    dataloader=get_data_loader(data, num_entities, _num_negative_samples, _batch_size, all_triples=all_data, data_type=data_type)
    
    final_metrics=defaultdict(float)
    dataset_metrics=[]
    
    with torch.no_grad():
        for batch, (positive_sample, corrupted_heads, corrupted_tails, filter_bias) in enumerate(dataloader):
            head_bias, tail_bias = filter_bias
            if torch.cuda.is_available():
                positive_sample=positive_sample.cuda()
                corrupted_heads=corrupted_heads.cuda()
                corrupted_tails=corrupted_tails.cuda()
                head_bias=head_bias.cuda()
                tail_bias=tail_bias.cuda()
                
                
            corrupted_head_dist=model((positive_sample, corrupted_heads), 'negative-head') + head_bias
            corrupted_tail_dist=model((positive_sample, corrupted_tails), 'negative-tail') + tail_bias
            
            head_arg_order=torch.argsort(corrupted_head_dist, dim=1, descending=True)
            tail_arg_order=torch.argsort(corrupted_tail_dist, dim=1, descending=True)
            
            true_head=positive_sample[:,0]
            true_tail=positive_sample[:,2]
            
            for ind in range(len(true_head)):
                true_head_rank=(head_arg_order[ind,:] == true_head[ind]).nonzero()
                true_head_rank = true_head_rank.item() + 1
                
                true_tail_rank = (tail_arg_order[ind,:] == true_tail[ind]).nonzero()
                true_tail_rank = true_tail_rank.item() + 1
                
                dataset_metrics.append({
                    'MRR': 1.0/true_head_rank,
                    'MR':float(true_head_rank),
                    'HITS@1':1.0 if true_head_rank <=1 else 0.0,
                    'HITS@3':1.0 if true_head_rank <=3 else 0.0,
                    'HITS@10':1.0 if true_head_rank <=10 else 0.0
                    
                })
                
                dataset_metrics.append({
                    'MRR': 1.0/true_tail_rank,
                    'MR':float(true_tail_rank),
                    'HITS@1':1.0 if true_tail_rank <=1 else 0.0,
                    'HITS@3':1.0 if true_tail_rank <=3 else 0.0,
                    'HITS@10':1.0 if true_tail_rank <=10 else 0.0
                    
                })
                
        for metric in dataset_metrics:
            for key, val in metric.items():
                final_metrics[key] += val
        
        for key,val in final_metrics.items():
            final_metrics[key] = val/len(dataset_metrics)
        
        print(f'{data_type} metrics: {final_metrics}')
            
            

In [46]:
def main():
    _num_negative_samples=128
    _batch_size=1024
    _test_batch_size=16
    _lr=0.0001
    _hidden_dim=1000
    _margin=12
    _num_epochs=1
    _weight_decay=5e-5
    _cuda=torch.cuda.is_available()
    
    entity2id = load_dict(os.path.join(DATA_DIR, 'entities.dict'))
    relation2id = load_dict(os.path.join(DATA_DIR, 'relations.dict'))
    num_entities=len(entity2id)
    num_relations=len(relation2id)
    
    train_data=load_triples(os.path.join(DATA_DIR, 'train.txt'), entity2id, relation2id)
    eval_data=load_triples(os.path.join(DATA_DIR, 'valid.txt'), entity2id, relation2id)
    test_data=load_triples(os.path.join(DATA_DIR, 'test.txt'), entity2id, relation2id)
    print(f'train length {len(train_data)}')
    print(f'eval length {len(eval_data)}')
    print(f'test length {len(test_data)}')
    all_data=[train_data, eval_data, test_data]
    
    dataloader = get_data_loader(train_data, num_entities, _num_negative_samples, _batch_size, data_type='train')
    
    model=KGEmbedding(num_entities, num_relations, _hidden_dim, _margin, 'transE')
    if _cuda:
        model=model.cuda()
        
    optimizer=torch.optim.Adam(filter(lambda p:p.requires_grad, model.parameters()), lr=_lr, weight_decay= _weight_decay)
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)
    
    print('Start Training...')
    for epoch in trange(_num_epochs, desc='Train', unit='Epoch'):
        total_loss=0
        model.train()
        optimizer.zero_grad()
        print('enumerate length ',enumerate(dataloader))
        for batch, (positive_sample, corrupted_heads, corrupted_tails, _) in enumerate(dataloader):
            if _cuda:
                positive_sample=positive_sample.cuda()
                corrupted_heads=corrupted_heads.cuda()
                corrupted_tails=corrupted_tails.cuda()
                
            positive_sample_dist=model(positive_sample, 'positive')
            positive_score=F.logsigmoid(positive_sample_dist)
            positive_sample_loss=-positive_score.mean()
            
            corrupted_head_dist=model((positive_sample, corrupted_heads), 'negative-head')
            corrupted_head_score=F.logsigmoid(-corrupted_head_dist)
            corrupted_head_loss=-corrupted_head_score.mean()
            
            corrupted_tail_dist=model((positive_sample, corrupted_tails),'negative-tail')
            corrupted_tail_score=F.logsigmoid(-corrupted_tail_dist)
            corrupted_tail_loss=-corrupted_tail_score.mean()
            
            loss=(positive_sample_loss + corrupted_head_loss + positive_sample_loss + corrupted_tail_loss)/4
            total_loss+=loss.item()
            
            print(f'\nbatch: {batch}, loss: {loss}, pos_loss: {positive_sample_loss}, neg_head_loss:{corrupted_head_loss}, neg_tail_loss: {corrupted_tail_loss}')
            
            loss.backward()
            optimizer.step()
    
        scheduler.step(total_loss)
        print(f'\nepoch: {epoch}, avg loss:{total_loss/len(dataloader)}')
        
       
        if epoch==0:
            eval_model(model,eval_data, num_entities, _num_negative_samples, _test_batch_size, all_data, 'eval')
            save_model(model, optimizer, scheduler, epoch)
        
        
    eval_model(model, test_data, num_entities, _num_negative_samples, _test_batch_size, all_data, 'test')

In [47]:
main()

train length 272115
eval length 17535
test length 20466
Start Training...


Train:   0%|                                           | 0/1 [00:00<?, ?Epoch/s]

enumerate length  <enumerate object at 0x7fb3654625a0>


[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)



batch: 0, loss: 0.7465620040893555, pos_loss: 0.4462284445762634, neg_head_loss:1.043308973312378, neg_tail_loss: 1.0504820346832275

batch: 1, loss: 0.7308329343795776, pos_loss: 0.46657195687294006, neg_head_loss:0.9965364336967468, neg_tail_loss: 0.9936514496803284

batch: 2, loss: 0.7192634344100952, pos_loss: 0.49911510944366455, neg_head_loss:0.944808840751648, neg_tail_loss: 0.9340145587921143

batch: 3, loss: 0.7088199853897095, pos_loss: 0.5212148427963257, neg_head_loss:0.9046688675880432, neg_tail_loss: 0.8881813883781433

batch: 4, loss: 0.7010917663574219, pos_loss: 0.5516703724861145, neg_head_loss:0.8580695390701294, neg_tail_loss: 0.8429567813873291

batch: 5, loss: 0.6913243532180786, pos_loss: 0.5683934092521667, neg_head_loss:0.825359046459198, neg_tail_loss: 0.803151547908783

batch: 6, loss: 0.6860454678535461, pos_loss: 0.5988953113555908, neg_head_loss:0.7852234244346619, neg_tail_loss: 0.7611677646636963

batch: 7, loss: 0.6793630123138428, pos_loss: 0.62100201

...


batch: 242, loss: 0.6669755578041077, pos_loss: 1.209039330482483, neg_head_loss:0.15471197664737701, neg_tail_loss: 0.0951116681098938

batch: 243, loss: 0.6546593308448792, pos_loss: 1.1813890933990479, neg_head_loss:0.15573805570602417, neg_tail_loss: 0.10012094676494598

batch: 244, loss: 0.6765038967132568, pos_loss: 1.2305793762207031, neg_head_loss:0.151437908411026, neg_tail_loss: 0.09341902285814285

batch: 245, loss: 0.7000672221183777, pos_loss: 1.2844845056533813, neg_head_loss:0.14036500453948975, neg_tail_loss: 0.09093496203422546

batch: 246, loss: 0.7012750506401062, pos_loss: 1.2887495756149292, neg_head_loss:0.13314883410930634, neg_tail_loss: 0.09445218741893768

batch: 247, loss: 0.6161136627197266, pos_loss: 1.116081953048706, neg_head_loss:0.135354682803154, neg_tail_loss: 0.09693627059459686

batch: 248, loss: 0.7013925313949585, pos_loss: 1.2919912338256836, neg_head_loss:0.1349545121192932, neg_tail_loss: 0.08663323521614075

batch: 249, loss: 0.63726973533630

[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


eval metrics: defaultdict(<class 'float'>, {'MRR': 0.10395888808448048, 'MR': 1269.6559167379526, 'HITS@1': 0.05161106358711149, 'HITS@3': 0.11522668947818648, 'HITS@10': 0.1956658112346735})


Train: 100%|█████████████████████████████████| 1/1 [25:12<00:00, 1512.64s/Epoch]
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


test metrics: defaultdict(<class 'float'>, {'MRR': 0.10042147524877144, 'MR': 1282.1407211961302, 'HITS@1': 0.04871494185478354, 'HITS@3': 0.11074464966285547, 'HITS@10': 0.1909508453044073})


In [54]:
data_relation=np.load(os.path.join(MODEL_DIR, f'relation_embedding_{0}.npy'))
print(data_relation.shape)             

data_entity=np.load(os.path.join(MODEL_DIR, f'entity_embedding_{0}.npy'))        
print(data_entity.shape)

# train length 272115
# eval length 17535
# test length 20466

# relations 237 
# entities 14541 
# hidden_dim 1000 




(237, 1000)
(14541, 1000)


In [ ]:
save_model(model, optimizer, scheduler, epoch)

In [ ]:
eval_model(model, test_data, num_entities, _num_negative_samples, _test_batch_size, all_data, 'test')